This notebook is a part of the work 'Nucleation patterns of polymer crystals analyzed by machine learning models' and is written by Atmika Bhardwaj (bhardwaj@ipfdd.de).

The following machine learning (ML) models are employed: AutoEncoders (AE), Hierarchical Clustering (HC), Gaussian Mixture Models (GMM), Multilayer Perceptron (MLP) to harness the capabilities of ML algorithms to unfold the details associated with the different phases that emerge during polymer crystallization. We start by reading a coordinate file (dump file from LAMMPS) and analyze it to study the local environmental information of every coarse-grained bead. Then, we train an AE to compress that information and train a GMM on the compressed data to classify each coarse-grained into amorphous or crytalline depending on its enviromental fingerprint.

This notebook imports all the functions from another file called all_functions.py


In [ ]:
import glob
import random
import pickle
import keras
import numpy                  as     np
from   numpy.random           import seed
import matplotlib.pyplot      as     plt
import matplotlib.cm          as     cm
import h5py                   as     h5py
from   pathlib                import Path
from   mpl_toolkits.mplot3d   import Axes3D
from   natsort                import natsorted
import sys
from   all_functions          import *
import matplotlib.ticker      as mticker

import our_colors; ocol=our_colors.generate_colors()
olab = our_colors.generate_labels()

import control_parameters
param = control_parameters.get_defaults()

F11min = param["F10min"]
F11max = param["F10max"]

seed(1)
plt.rcParams["figure.figsize"]  = (6,5)
plt.rcParams["figure.dpi"]      = 300
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

In [ ]:
g           = natsorted(glob.glob('../../01_raw_data/dump_files/dump.*'))
file_num    = -1
obj         = ReadCoordinates(g[file_num], param["chainlength"])
rnd         = np.arange(obj.totalAtoms)
print('Total atoms = ', obj.totalAtoms)

In [ ]:
filename       = '../../02_processed_data/Labels/AE_labels.sav'
ae_labels      = np.array(pickle.load(open(filename, 'rb')))[file_num]
filename       = '../../02_processed_data/Labels/SL_labels.sav'
sl_labels      = np.array(pickle.load(open(filename, 'rb')))[file_num]
filename       = '../../02_processed_data/Labels/AO_labels.sav'
avgor_lab      = np.array(pickle.load(open(filename, 'rb')))[file_num]


In [ ]:
grid = GridSearch(obj, numSigma = 6.0)
grid.updateNeighborLists(obj, rnd, param["Rmax"])


In [ ]:
def crystal_environment(labels, gridobj):
    frac     = []
    crys_lab = np.where(labels == 1)[0]
    for i in crys_lab:
        indx = gridobj.indices[i]
        temp = np.sum(labels[indx] == 1)
        frac.append(temp/len(indx))
    return np.array(frac)

def scatter_hist(frac_arr):
    num, bins = np.histogram(frac_arr, bins = np.arange(0, 1, 0.05))
    X         = (bins[:-1] + bins[1:])/2
    Y         = num
    return X, Y

In [ ]:
frac_ae  = crystal_environment(ae_labels, grid)
frac_sl  = crystal_environment(sl_labels, grid)
frac_avg = crystal_environment(avgor_lab, grid)

In [ ]:
marker_size = 6
line_wid    = 2

data= []
fig = plt.figure()

X, Y = scatter_hist(frac_ae)
plt.plot(X, Y/sum(Y), label = olab["sAE"], ms = marker_size, c = ocol["AE"],   marker="s", lw = line_wid)
data = np.c_[X, Y]

X, Y = scatter_hist(frac_sl)
plt.plot(X, Y/sum(Y), label = olab["sSL"], ms = marker_size, c = ocol["SL"], marker="v", lw = line_wid)
data = np.c_[data, X, Y]

X, Y = scatter_hist(frac_avg)
plt.plot(X, Y/sum(Y), label = olab["sAO"], ms = marker_size, c = ocol["AO"], marker = "o", lw = line_wid)
data = np.c_[data, X, Y]

np.savetxt('../../03_figures/FigS4/FigS4a.csv', data, delimiter=",")

lgnd     = plt.legend(fontsize = 22)
plt.xlabel('$ f_{Cryst}$', fontsize = 25)
plt.ylabel('Frequency', fontsize = 20)
plt.grid()
plt.tight_layout()
plt.savefig('../../03_figures/FigS4/FigS4a.png',bbox_inches="tight")


In [ ]:
avg_ae  = np.mean(frac_ae)
avg_sl  = np.mean(frac_sl)
avg_avg = np.mean(frac_avg)
std_ae  = np.std(frac_ae)/  np.sqrt(np.sum(ae_labels)/15)
std_sl  = np.std(frac_sl)/  np.sqrt(np.sum(sl_labels)/15)
std_avg = np.std(frac_avg)/ np.sqrt(np.sum(avgor_lab)/15)
labels  = ['AE labels', 'SL labels', 'AO labels']
axislabels  = [olab["sAE"], olab["sSL"], olab["sAO"]]

x_pos   = np.arange(len(axislabels))
means   = [avg_ae, avg_sl, avg_avg]
error   = [std_ae, std_sl, std_avg]

In [ ]:
fig, ax = plt.subplots(figsize = (6,5))
np.savetxt('../../03_figures/FigS4/FigS4b.csv', [x_pos, means, error], delimiter=",")
keys    = ["AE","SL","AO"]
kcol    = [ocol[keys[i]] for i in range(3)]
print (kcol)

MS      = 12
ax.errorbar(0, means[0], yerr = error[0], ms = MS, marker = "o", c = ocol["AE"], ls="",capsize = 10)
ax.errorbar(1, means[1], yerr = error[1], ms = MS, marker = "o", c = ocol["SL"], ls="",capsize = 10)
ax.errorbar(2, means[2], yerr = error[2], ms = MS, marker = "o", c = ocol["AO"], ls="",capsize = 10)

ax.set_xlabel     ('Classification method', fontsize = 20)
ax.tick_params    (axis='x', rotation=90)
ax.set_ylabel     ('Average $ f_{Cryst}$', fontsize = 22)
ax.set_xticks     (x_pos)
ax.set_xticklabels(axislabels, fontsize = 20, rotation = 0 )
ax.tick_params    (axis='y', labelsize=18)
ax.yaxis.grid     (True)
ax.xaxis.grid     (True)
plt.tight_layout  ()
plt.savefig('../../03_figures/FigS4/FigS4b.png',bbox_inches="tight")

## Decay of newcomers

### ML labels

In [ ]:
filename       = '../../02_processed_data/Labels/AE_labels.sav'
all_ae_labels  = np.array(pickle.load(open(filename, 'rb')))

time_step          = 40
dt                 = 1
diff_new_lost_crys = []
times              = np.arange(time_step, len(g)- 4*dt - 1, dt)
all_loses          = []
for timestep in times:
    aelabelt      = all_ae_labels[timestep]
    aelabelt_plus = all_ae_labels[timestep + dt]
    ae1_t         = np.where(aelabelt      == 1)[0]
    ae1_t_plus    = np.where(aelabelt_plus == 1)[0]               #total crystallinity = shared + found
    ae0_t_plus    = np.where(aelabelt_plus == 0)[0]
    #commonidx     = np.intersect1d(ae1_t,      ae1_t_plus)       #shared between t and t+dt, surviving
    amor_t_plus   = np.intersect1d(ae1_t,      ae0_t_plus)        #crystalline at t, became amorphous at t+dt, decay
    
    new_comers    = np.setdiff1d  (ae1_t_plus, ae1_t)             #newcomers at t+dt
    print(timestep)
    print('ae_T = ', len(ae1_t),    'ae_T+dt = ', len(ae1_t_plus), 'newcomers = ', len(new_comers),
          'lost = ', len(amor_t_plus))
    
    lost_idx      = []
                                                                  #decay from t+dt to t+dt+Delta t
    for i in np.arange(timestep + dt + 1, len(g)):                #after timestep till the end
        aelabelt  = all_ae_labels[i]
        ae0_t     = np.where(aelabelt == 0)[0]
        lost_idx.append(len(np.intersect1d(new_comers, ae0_t)))   #crystalline newcomers become amorphous with time  
    diff_new_lost_crys.append([len(ae1_t_plus) - len(ae1_t), len(new_comers), 
                          lost_idx[0], lost_idx[2], lost_idx[-3], lost_idx[-1], len(ae1_t), len(ae1_t_plus)])
    all_loses.append(np.array(lost_idx))

diff_new_lost_crys = np.array(diff_new_lost_crys)



In [ ]:
gain     = diff_new_lost_crys[:, 0]
replaced = diff_new_lost_crys[:, 1] - gain 
all_loses

In [ ]:
marker_size = 10
fig         = plt.figure(figsize=(6,7))

tlabel_y    = 0.173
tlabel_dx   = 0.5
tmin, tmax  = 44,72

ax          = fig.add_subplot()
ax2         = ax.twiny()
temp        = np.linspace(0.75, 0.9, 75)[1:][time_step:time_step+len(diff_new_lost_crys)]
ax2.tick_params(axis='x', labelsize=19)
ax2.set_xlabel('Temperature (LJ-units)', fontsize = 20, labelpad = 11)

np.savetxt('../../03_figures/Fig12/Fig12a.csv', diff_new_lost_crys, delimiter=",")

ax.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '')
ax.axvline(param["t_tr"] , ymax=0.66, color = ocol["t_tr"],   ls = '--', lw = 2, label = '')
ax.axvline(param["t_min"], ymax=0.66, color = ocol["t_min"],  ls = '--', lw = 2, label = '')

ax.text(param["t_pre"]+tlabel_dx, tlabel_y, '$t_{pre}$', color = ocol["t_pre"], fontsize=15)
ax.text(param["t_tr"] +tlabel_dx, tlabel_y, '$t_{tr}$',  color = ocol["t_tr"] , fontsize=15)
ax.text(param["t_min"]+tlabel_dx, tlabel_y, '$t_{min}$', color = ocol["t_min"], fontsize=15)

factor = 0.5
def greyed(index):
    grey=0.6*np.ones(3)
    return grey + (ocol["AE"]-grey)*(1-index/3.0)

print(dt,times+1,diff_new_lost_crys[:,2]/diff_new_lost_crys[:,1],diff_new_lost_crys[:,2])

ax.plot(times+dt+1, diff_new_lost_crys[:,2]/diff_new_lost_crys[:,1], color = greyed(0), ms = marker_size, marker = "o", label = olab["sAE"]+' $ \Delta t $ = 1')
ax.plot(times+dt+1, diff_new_lost_crys[:,3]/diff_new_lost_crys[:,1], color = greyed(1), ms = marker_size, marker = "v", label = olab["sAE"]+' $ \Delta t $ = 3')
ax.plot(times+dt+1, diff_new_lost_crys[:,4]/diff_new_lost_crys[:,1], color = greyed(2), ms = marker_size, marker = "^", label = olab["sAE"]+' $ \Delta t $ = $ t_{end} - 2 - t$')
ax.plot(times+dt+1, diff_new_lost_crys[:,5]/diff_new_lost_crys[:,1], color = greyed(3), ms = marker_size, marker = "s", label = olab["sAE"]+' $ \Delta t $ = $ t_{end} - t$')

ax.set_xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
ax.set_ylabel('$ f_{d_{NC}} (t, t + \Delta t)$', fontsize = 25)
ax.tick_params(axis='both', labelsize=19)
ax.legend(fontsize = 17, loc = 'best')

ax.set_xlim(tmin, tmax)
ax2.set_xlim(control_parameters.calcLJTempAtTime(tmin), control_parameters.calcLJTempAtTime(tmax))

plt.ylim(0.15, 1.02)
plt.savefig('../../03_figures/Fig12/Fig12a.png',bbox_inches="tight")


### AO lables

In [ ]:
filename       = '../../02_processed_data/Labels/AO_labels2.sav'
all_ao_labels  = np.array(pickle.load(open(filename, 'rb')))

time_step          = 40
dt                 = 1
diff_new_lost_crys = []
times              = np.arange(time_step, len(g)- 4*dt - 1, dt)
all_loses          = []
for timestep in times:
    aolabelt      = all_ao_labels[timestep]
    aolabelt_plus = all_ao_labels[timestep + dt]
    ao1_t         = np.where(aolabelt      == 1)[0]
    ao1_t_plus    = np.where(aolabelt_plus == 1)[0]               #total crystallinity = shared + found
    ao0_t_plus    = np.where(aolabelt_plus == 0)[0]
    #commonidx     = np.intersect1d(ao1_t,      ao1_t_plus)       #shared between t and t+dt, surviving
    amor_t_plus   = np.intersect1d(ao1_t,      ao0_t_plus)        #crystalline at t, became amorphous at t+dt, decay
    new_comers    = np.setdiff1d  (ao1_t_plus, ao1_t)             #newcomers at t+dt
    print(timestep)
    print('ao_T = ', len(ao1_t),    'ao_T+dt = ', len(ao1_t_plus), 'newcomers = ', len(new_comers),
          'lost = ', len(amor_t_plus))
    
    lost_idx      = []
    for i in np.arange(timestep + dt + 1, len(g)):                #after timestep till the end
        aolabelt  = all_ao_labels[i]
        ao0_t     = np.where(aolabelt == 0)[0]
        lost_idx.append(len(np.intersect1d(new_comers, ao0_t)))   #crystalline newcomers become amorphous with time  
    diff_new_lost_crys.append([len(ao1_t_plus) - len(ao1_t), len(new_comers), 
                          lost_idx[0], lost_idx[2], lost_idx[-3], lost_idx[-1], len(ao1_t), len(ao1_t_plus)])
    all_loses.append(np.array(lost_idx))

diff_new_lost_crys = np.array(diff_new_lost_crys)


In [ ]:
marker_size = 10
fig         = plt.figure(figsize=(6,7))

tlabel_y    = 0.173
tlabel_dx   = 0.5
tmin, tmax  = 44, 72
ax          = fig.add_subplot()
ax2         = ax.twiny()
temp        = np.linspace(0.9, 0.75, 75)[1:][time_step:time_step+len(diff_new_lost_crys)]

ax2.tick_params(axis='x', labelsize=19)
ax2.set_xlabel('Temperature (LJ-units)', fontsize = 20, labelpad = 11)

np.savetxt('../../03_figures/Fig12/Fig12c.csv', diff_new_lost_crys, delimiter=",")

ax.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '')
ax.axvline(param["t_tr"] , ymax=0.66, color = ocol["t_tr"],   ls = '--', lw = 2, label = '')
ax.axvline(param["t_min"], ymax=0.66, color = ocol["t_min"],  ls = '--', lw = 2, label = '')

ax.text(param["t_pre"]+tlabel_dx, tlabel_y, '$t_{pre}$', color = ocol["t_pre"], fontsize=15)
ax.text(param["t_tr"] +tlabel_dx, tlabel_y, '$t_{tr}$',  color = ocol["t_tr"] , fontsize=15)
ax.text(param["t_min"]+tlabel_dx, tlabel_y, '$t_{min}$', color = ocol["t_min"], fontsize=15)

factor=0.5
def greyed(index):
    grey=0.6*np.ones(3)
    return grey + (ocol["AO"]-grey)*(1-index/3.0)

select=diff_new_lost_crys[:,1]!=0

ax.plot(times[select]+dt+1, diff_new_lost_crys[:,2][select]/diff_new_lost_crys[:,1][select], color = greyed(0), ms = marker_size, marker = "o", label = olab["sAOII"]+' $ \Delta t $ = 1')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,3][select]/diff_new_lost_crys[:,1][select], color = greyed(1), ms = marker_size, marker = "v", label = olab["sAOII"]+' $ \Delta t $ = 3')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,4][select]/diff_new_lost_crys[:,1][select], color = greyed(2), ms = marker_size, marker = "^", label = olab["sAOII"]+' $ \Delta t $ = $ t_{end} - 2 - t$')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,5][select]/diff_new_lost_crys[:,1][select], color = greyed(3), ms = marker_size, marker = "s", label = olab["sAOII"]+' $ \Delta t $ = $ t_{end} - t$')

ax.set_xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
ax.set_ylabel('$ f_{d_{NC}} (t, t + \Delta t)$', fontsize = 25)
ax.tick_params(axis='both', labelsize=19)
ax.legend(fontsize = 17, loc = 'best')

ax.set_xlim(tmin, tmax)
ax2.set_xlim(control_parameters.calcLJTempAtTime(tmin), control_parameters.calcLJTempAtTime(tmax))

plt.ylim(0.15, 1.02)
plt.savefig('../../03_figures/Fig12/Fig12c.png',bbox_inches="tight")


In [ ]:
filename       = '../../02_processed_data/Labels/AO_labels.sav'
all_ao_labels  = np.array(pickle.load(open(filename, 'rb')))

time_step          = 40
dt                 = 1
diff_new_lost_crys = []
times              = np.arange(time_step, len(g)- 4*dt - 1, dt)
all_loses          = []
for timestep in times:
    aolabelt      = all_ao_labels[timestep]
    aolabelt_plus = all_ao_labels[timestep + dt]
    ao1_t         = np.where(aolabelt      == 1)[0]
    ao1_t_plus    = np.where(aolabelt_plus == 1)[0]               #total crystallinity = shared + found
    ao0_t_plus    = np.where(aolabelt_plus == 0)[0]
    #commonidx     = np.intersect1d(ao1_t,      ao1_t_plus)       #shared between t and t+dt, surviving
    amor_t_plus   = np.intersect1d(ao1_t,      ao0_t_plus)        #crystalline at t, became amorphous at t+dt, decay
    new_comers    = np.setdiff1d  (ao1_t_plus, ao1_t)             #newcomers at t+dt
    print(timestep)
    print('ao_T = ', len(ao1_t),    'ao_T+dt = ', len(ao1_t_plus), 'newcomers = ', len(new_comers),
          'lost = ', len(amor_t_plus))
    
    lost_idx      = []
    for i in np.arange(timestep + dt + 1, len(g)):                #after timestep till the end
        aolabelt  = all_ao_labels[i]
        ao0_t     = np.where(aolabelt == 0)[0]
        lost_idx.append(len(np.intersect1d(new_comers, ao0_t)))   #crystalline newcomers become amorphous with time  
    diff_new_lost_crys.append([len(ao1_t_plus) - len(ao1_t), len(new_comers), 
                          lost_idx[0], lost_idx[2], lost_idx[-3], lost_idx[-1], len(ao1_t), len(ao1_t_plus)])
    all_loses.append(np.array(lost_idx))

diff_new_lost_crys = np.array(diff_new_lost_crys)

In [ ]:
marker_size = 10
fig         = plt.figure(figsize=(6,7))

ax          = fig.add_subplot()
ax2         = ax.twiny()
temp        = np.linspace(0.9, 0.75, 75)[1:][time_step:time_step+len(diff_new_lost_crys)]

ax2.tick_params(axis='x', labelsize=19)
ax2.set_xlabel('Temperature (LJ-units)', fontsize = 20, labelpad = 11)

np.savetxt('../../03_figures/FigS9/FigS9b.csv', diff_new_lost_crys, delimiter=",")

ax.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '')
ax.axvline(param["t_tr"] , ymax=0.66, color = ocol["t_tr"],   ls = '--', lw = 2, label = '')
ax.axvline(param["t_min"], ymax=0.66, color = ocol["t_min"],  ls = '--', lw = 2, label = '')

ax.text(param["t_pre"]+tlabel_dx, tlabel_y, '$t_{pre}$', color = ocol["t_pre"], fontsize=15)
ax.text(param["t_tr"] +tlabel_dx, tlabel_y, '$t_{tr}$',  color = ocol["t_tr"] , fontsize=15)
ax.text(param["t_min"]+tlabel_dx, tlabel_y, '$t_{min}$', color = ocol["t_min"], fontsize=15)


factor = 0.5
def greyed(index):
    grey=0.6*np.ones(3)
    return grey + (ocol["AO"]-grey)*(1-index/3.0)

select=diff_new_lost_crys[:,1]!=0

ax.plot(times[select]+dt+1, diff_new_lost_crys[:,2][select]/diff_new_lost_crys[:,1][select], color = greyed(0), ms = marker_size, marker = "o", label = olab["sAO"]+' $ \Delta t $ = 1')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,3][select]/diff_new_lost_crys[:,1][select], color = greyed(1), ms = marker_size, marker = "v", label = olab["sAO"]+' $ \Delta t $ = 3')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,4][select]/diff_new_lost_crys[:,1][select], color = greyed(2), ms = marker_size, marker = "^", label = olab["sAO"]+' $ \Delta t $ = $ t_{end} - 2 - t$')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,5][select]/diff_new_lost_crys[:,1][select], color = greyed(3), ms = marker_size, marker = "s", label = olab["sAO"]+' $ \Delta t $ = $ t_{end} - t$')

ax.set_xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
ax.set_ylabel('$ f_{d_{NC}} (t, t + \Delta t)$', fontsize = 25)
ax.tick_params(axis='both', labelsize=19)
ax.legend(fontsize = 17, loc = 'best')

ax.set_xlim(tmin, tmax)
ax2.set_xlim(control_parameters.calcLJTempAtTime(tmin), control_parameters.calcLJTempAtTime(tmax))

plt.ylim(0.15, 1.02)
plt.savefig('../../03_figures/FigS9/FigS9b.png',bbox_inches="tight")

### SL labels

In [ ]:
filename       = '../../02_processed_data/Labels/SL_labels2.sav'
all_ao_labels  = np.array(pickle.load(open(filename, 'rb')))

time_step          = 40
dt                 = 1
diff_new_lost_crys = []
times              = np.arange(time_step, len(g)- 4*dt - 1, dt)
all_loses          = []
for timestep in times:
    aolabelt      = all_ao_labels[timestep]
    aolabelt_plus = all_ao_labels[timestep + dt]
    ao1_t         = np.where(aolabelt      == 1)[0]
    ao1_t_plus    = np.where(aolabelt_plus == 1)[0]               #total crystallinity = shared + found
    ao0_t_plus    = np.where(aolabelt_plus == 0)[0]
    #commonidx     = np.intersect1d(ao1_t,      ao1_t_plus)       #shared between t and t+dt, surviving
    amor_t_plus   = np.intersect1d(ao1_t,      ao0_t_plus)        #crystalline at t, became amorphous at t+dt, decay
    new_comers    = np.setdiff1d  (ao1_t_plus, ao1_t)             #newcomers at t+dt
    print(timestep)
    print('ao_T = ', len(ao1_t),    'ao_T+dt = ', len(ao1_t_plus), 'newcomers = ', len(new_comers),
          'lost = ', len(amor_t_plus))
    
    lost_idx      = []
    for i in np.arange(timestep + dt + 1, len(g)):                #after timestep till the end
        aolabelt  = all_ao_labels[i]
        ao0_t     = np.where(aolabelt == 0)[0]
        lost_idx.append(len(np.intersect1d(new_comers, ao0_t)))   #crystalline newcomers become amorphous with time  
    diff_new_lost_crys.append([len(ao1_t_plus) - len(ao1_t), len(new_comers), 
                          lost_idx[0], lost_idx[2], lost_idx[-3], lost_idx[-1], len(ao1_t), len(ao1_t_plus)])
    all_loses.append(np.array(lost_idx))

diff_new_lost_crys = np.array(diff_new_lost_crys)

In [ ]:
marker_size = 10
fig         = plt.figure(figsize=(6,7))
ax          = fig.add_subplot()
ax2         = ax.twiny()
temp        = np.linspace(0.9, 0.75, 75)[1:][time_step:time_step+len(diff_new_lost_crys)]

ax2.tick_params(axis='x', labelsize=19)
ax2.set_xlabel('Temperature (LJ-units)', fontsize = 20, labelpad = 11)

np.savetxt('../../03_figures/Fig12/Fig12b.csv', diff_new_lost_crys, delimiter=",")

data_fig21 = np.loadtxt('../../03_figures/FigS9/FigS9a.csv', delimiter=",")

ax.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '')
ax.axvline(param["t_tr"] , ymax=0.62, color = ocol["t_tr"],   ls = '--', lw = 2, label = '')
ax.axvline(param["t_min"], ymax=0.62, color = ocol["t_min"],  ls = '--', lw = 2, label = '')

ax.text(param["t_pre"]+tlabel_dx, tlabel_y, '$t_{pre}$', color = ocol["t_pre"], fontsize=15)
ax.text(param["t_tr"] +tlabel_dx, tlabel_y, '$t_{tr}$',  color = ocol["t_tr"] , fontsize=15)
ax.text(param["t_min"]+tlabel_dx, tlabel_y, '$t_{min}$', color = ocol["t_min"], fontsize=15)


factor = 0.5
def greyed(index):
    grey = 0.6*np.ones(3)
    return grey + (ocol["SL"]-grey)*(1-index/3.0)

select=diff_new_lost_crys[:,1]!=0
select_fig21=diff_new_lost_crys[:,1]!=0

ax.plot(times[select_fig21]+dt+1, data_fig21[:,2][select_fig21]/data_fig21[:,1][select_fig21],color = ocol["t_end"], linestyle="dotted", ms = marker_size, fillstyle="none", marker = "o",  label = olab["sSL"]+' $ \Delta t $ = 1', alpha=0.5)
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,2][select]/diff_new_lost_crys[:,1][select], color = greyed(0), ms = marker_size, marker = "o", label = olab["sSLII"]+' $ \Delta t $ = 1')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,3][select]/diff_new_lost_crys[:,1][select], color = greyed(1), ms = marker_size, marker = "v", label = olab["sSLII"]+' $ \Delta t $ = 3')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,4][select]/diff_new_lost_crys[:,1][select], color = greyed(2), ms = marker_size, marker = "^", label = olab["sSLII"]+' $ \Delta t $ = $ t_{end} - 2 - t$')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,5][select]/diff_new_lost_crys[:,1][select], color = greyed(3), ms = marker_size, marker = "s", label = olab["sSLII"]+' $ \Delta t $ = $ t_{end} - t$')

ax.set_xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
ax.set_ylabel('$ f_{d_{NC}} (t, t + \Delta t)$', fontsize = 25)
ax.tick_params(axis='both', labelsize=19)
ax.legend(fontsize = 17, loc = 'best')

ax.set_xlim(tmin, tmax)
ax2.set_xlim(control_parameters.calcLJTempAtTime(tmin), control_parameters.calcLJTempAtTime(tmax))

plt.ylim(0.15, 1.02)
plt.savefig('../../03_figures/Fig12/Fig12b.png',bbox_inches="tight")

In [ ]:
filename       = '../../02_processed_data/Labels/SL_labels.sav'
all_ao_labels  = np.array(pickle.load(open(filename, 'rb')))

time_step          = 40
dt                 = 1
diff_new_lost_crys = []
times              = np.arange(time_step, len(g)- 4*dt - 1, dt)
all_loses          = []
for timestep in times:
    aolabelt      = all_ao_labels[timestep]
    aolabelt_plus = all_ao_labels[timestep + dt]
    ao1_t         = np.where(aolabelt      == 1)[0]
    ao1_t_plus    = np.where(aolabelt_plus == 1)[0]               #total crystallinity = shared + found
    ao0_t_plus    = np.where(aolabelt_plus == 0)[0]
    #commonidx     = np.intersect1d(ao1_t,      ao1_t_plus)       #shared between t and t+dt, surviving
    amor_t_plus   = np.intersect1d(ao1_t,      ao0_t_plus)        #crystalline at t, became amorphous at t+dt, decay
    new_comers    = np.setdiff1d  (ao1_t_plus, ao1_t)             #newcomers at t+dt
    print(timestep)
    print('ao_T = ', len(ao1_t),    'ao_T+dt = ', len(ao1_t_plus), 'newcomers = ', len(new_comers),
          'lost = ', len(amor_t_plus))
    
    lost_idx      = []
    for i in np.arange(timestep + dt + 1, len(g)):                #after timestep till the end
        aolabelt  = all_ao_labels[i]
        ao0_t     = np.where(aolabelt == 0)[0]
        lost_idx.append(len(np.intersect1d(new_comers, ao0_t)))   #crystalline newcomers become amorphous with time  
    diff_new_lost_crys.append([len(ao1_t_plus) - len(ao1_t), len(new_comers), 
                          lost_idx[0], lost_idx[2], lost_idx[-3], lost_idx[-1], len(ao1_t), len(ao1_t_plus)])
    all_loses.append(np.array(lost_idx))

diff_new_lost_crys = np.array(diff_new_lost_crys)

In [ ]:
marker_size = 10
fig         = plt.figure(figsize=(6,7))

ax          = fig.add_subplot()
ax2         = ax.twiny()
temp        = np.linspace(0.9, 0.75, 75)[1:][time_step:time_step+len(diff_new_lost_crys)]

ax2.tick_params(axis='x', labelsize=19)
ax2.set_xlabel('Temperature (LJ-units)', fontsize = 20, labelpad = 11)

np.savetxt('../../03_figures/FigS9/FigS9a.csv', diff_new_lost_crys, delimiter=",")

ax.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '')
ax.axvline(param["t_tr"] , ymax=0.66, color = ocol["t_tr"],   ls = '--', lw = 2, label = '')
ax.axvline(param["t_min"], ymax=0.66, color = ocol["t_min"],  ls = '--', lw = 2, label = '')

ax.text(param["t_pre"]+tlabel_dx, tlabel_y, '$t_{pre}$', color = ocol["t_pre"], fontsize=15)
ax.text(param["t_tr"] +tlabel_dx, tlabel_y, '$t_{tr}$',  color = ocol["t_tr"] , fontsize=15)
ax.text(param["t_min"]+tlabel_dx, tlabel_y, '$t_{min}$', color = ocol["t_min"], fontsize=15)


factor=0.5
def greyed(index):
    grey=0.6*np.ones(3)
    return grey + (ocol["SL"]-grey)*(1-index/3.0)

select=diff_new_lost_crys[:,1]!=0

ax.plot(times[select]+dt+1, diff_new_lost_crys[:,2][select]/diff_new_lost_crys[:,1][select], color = greyed(0), ms = marker_size, marker = "o", label = olab["sSL"]+' $ \Delta t $ = 1')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,3][select]/diff_new_lost_crys[:,1][select], color = greyed(1), ms = marker_size, marker = "v", label = olab["sSL"]+' $ \Delta t $ = 3')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,4][select]/diff_new_lost_crys[:,1][select], color = greyed(2), ms = marker_size, marker = "^", label = olab["sSL"]+' $ \Delta t $ = $ t_{end} - 2 - t$')
ax.plot(times[select]+dt+1, diff_new_lost_crys[:,5][select]/diff_new_lost_crys[:,1][select], color = greyed(3), ms = marker_size, marker = "s", label = olab["sSL"]+' $ \Delta t $ = $ t_{end} - t$')

ax.set_xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
ax.set_ylabel('$ f_{d_{NC}} (t, t + \Delta t)$', fontsize = 25)
ax.tick_params(axis='both', labelsize=19)
ax.legend(fontsize = 17, loc = 'best')

ax.set_xlim(tmin, tmax)
ax2.set_xlim(control_parameters.calcLJTempAtTime(tmin), control_parameters.calcLJTempAtTime(tmax))

plt.ylim(0.15, 1.02)
plt.savefig('../../03_figures/FigS9/FigS9a.png',bbox_inches="tight")

### Interscheme comparison

In [ ]:
time_step  = -1
filename   = '../../02_processed_data/Labels/AE_labels.sav'
ae_labels  = np.array(pickle.load(open(filename, 'rb')))[time_step]
filename   = '../../02_processed_data/Labels/SL_labels.sav'
sl_labels  = np.array(pickle.load(open(filename, 'rb')))[time_step]
filename   = '../../02_processed_data/Labels/AO_labels.sav'
ao_labels  = np.array(pickle.load(open(filename, 'rb')))[time_step]

In [ ]:
ae1 = np.where(ae_labels  == 1)[0]
sl1 = np.where(sl_labels  == 1)[0]
ao1 = np.where(ao_labels  == 1)[0]

#ML
print('ML vs SL: ', sum(np.in1d(ae1, sl1))/len(ae1))
print('ML vs AO: ', sum(np.in1d(ae1, ao1))/len(ae1))
print()

#SL
print('SL vs ML: ', sum(np.in1d(sl1, ae1))/len(sl1))
print('SL vs AO: ', sum(np.in1d(sl1, ao1))/len(sl1))
print()

#AO
print('AO vs ML: ', sum(np.in1d(ao1, ae1))/len(ao1))
print('AO vs SL: ', sum(np.in1d(ao1, sl1))/len(ao1))


### Points selection criterion

In [ ]:
pathlist = Path('../../02_processed_data/Training_data/Other_Dataset/').glob('Random*/AE_labels.sav')
for path in pathlist:
    # because path is object not string
    path_in_str = str(path)
    print(path_in_str)

In [ ]:
filename   = '../../02_processed_data/Training_data/Other_Dataset/Randomslice15/AE_labels.sav'
ae_labels  = np.array(pickle.load(open(filename, 'rb')))[-1]

In [ ]:
def weird_division(n, d):
    if len(d):
        return sum(np.in1d(d, n))/len(d)
    else: return 0

pathlist = Path('../../02_processed_data/Training_data/Other_Dataset/').glob('Random*/AE_labels.sav')
common   = []
len_crys     = []
for path in pathlist:
    # because path is object not string
    filename   = str(path)
    print(filename)
    labe1s_new = np.array(pickle.load(open(filename, 'rb')))[-1]
    ae_old     = np.where(ae_labels  == 1)[0]
    ae_new     = np.where(labe1s_new == 1)[0]
    len_crys.append(len(ae_new))
    #p_ff.append(weird_division(ae_old, ae_new))
    common.append(sum(np.in1d(ae_old, ae_new)))
    #print(np.std(np.array([len(ae_old), random_common[i]]) )/len(ae_old))
common = np.array(common)
len_crys    = np.array(len_crys)

In [ ]:
common

In [ ]:
common, len_crys, len(ae_old)

In [ ]:
deviation = len(ae_old) - common
deviation

### Comparing point 10k

In [ ]:
timestep   = -1
filename   = '../../02_processed_data/Training_data/Other_Dataset/Randompoints10k/AE_labels.sav'
ae_labels  = np.array(pickle.load(open(filename, 'rb')))[timestep]
#frac_ae    = crystal_environment(ae_labels, grid)

pathlist = Path('../../02_processed_data/Training_data/Other_Dataset/').glob('Random*/AE_labels.sav')
common  = []
len_crys     = []
for path in pathlist:
    # because path is object not string
    filename   = str(path)
    print(filename)
    labe1s_new = np.array(pickle.load(open(filename, 'rb')))[timestep]
    ae_old     = np.where(ae_labels  == 1)[0]
    ae_new     = np.where(labe1s_new == 1)[0]
    len_crys.append(len(ae_new))
    #p_ff.append(weird_division(ae_old, ae_new))
    common.append(sum(np.in1d(ae_old, ae_new)))
    #print(np.std(np.array([len(ae_old), random_common[i]]) )/len(ae_old))
common = np.array(common)
len_crys    = np.array(len_crys)

np.std(np.array([len(ae_old), common], dtype=object))/len(ae_old) * 100

### Comparing slice 14

In [ ]:
timestep   = -1
filename   = '../../02_processed_data/Training_data/Other_Dataset/Randomslice14/AE_labels.sav'
ae_labels  = np.array(pickle.load(open(filename, 'rb')))[timestep]
#frac_ae    = crystal_environment(ae_labels, grid)

pathlist = Path('../../02_processed_data/Training_data/Other_Dataset/').glob('Random*/AE_labels.sav')
common  = []
len_crys     = []
for path in pathlist:
    # because path is object not string
    filename   = str(path)
    print(filename)
    labe1s_new = np.array(pickle.load(open(filename, 'rb')))[timestep]
    ae_old     = np.where(ae_labels  == 1)[0]
    ae_new     = np.where(labe1s_new == 1)[0]
    len_crys.append(len(ae_new))
    #p_ff.append(weird_division(ae_old, ae_new))
    common.append(sum(np.in1d(ae_old, ae_new)))
    #print(np.std(np.array([len(ae_old), random_common[i]]) )/len(ae_old))
common = np.array(common)
len_crys    = np.array(len_crys)

np.std(np.array([len(ae_old), common], dtype=object))/len(ae_old) * 100

### Comparing slice 15 

In [ ]:
timestep   = -1
filename   = '../../02_processed_data/Training_data/Other_Dataset/Randomslice15/AE_labels.sav'
ae_labels  = np.array(pickle.load(open(filename, 'rb')))[timestep]
#frac_ae    = crystal_environment(ae_labels, grid)

pathlist = Path('../../02_processed_data/Training_data/Other_Dataset/').glob('Random*/AE_labels.sav')
common  = []
len_crys     = []
for path in pathlist:
    # because path is object not string
    filename   = str(path)
    print(filename)
    labe1s_new = np.array(pickle.load(open(filename, 'rb')))[timestep]
    ae_old     = np.where(ae_labels  == 1)[0]
    ae_new     = np.where(labe1s_new == 1)[0]
    len_crys.append(len(ae_new))
    #p_ff.append(weird_division(ae_old, ae_new))
    common.append(sum(np.in1d(ae_old, ae_new)))
    #print(np.std(np.array([len(ae_old), random_common[i]]) )/len(ae_old))
common = np.array(common)
len_crys    = np.array(len_crys)

np.std(np.array([len(ae_old), common], dtype=object))/len(ae_old) * 100

### Comparing slice 56

In [ ]:
timestep   = -1
filename   = '../../02_processed_data/Training_data/Other_Dataset/Randomslice56/AE_labels.sav'
ae_labels  = np.array(pickle.load(open(filename, 'rb')))[timestep]
#frac_ae    = crystal_environment(ae_labels, grid)

pathlist = Path('../../02_processed_data/Training_data/Other_Dataset/').glob('Random*/AE_labels.sav')
common  = []
len_crys     = []
for path in pathlist:
    # because path is object not string
    filename   = str(path)
    print(filename)
    labe1s_new = np.array(pickle.load(open(filename, 'rb')))[timestep]
    ae_old     = np.where(ae_labels  == 1)[0]
    ae_new     = np.where(labe1s_new == 1)[0]
    len_crys.append(len(ae_new))
    #p_ff.append(weird_division(ae_old, ae_new))
    common.append(sum(np.in1d(ae_old, ae_new)))
    #print(np.std(np.array([len(ae_old), random_common[i]]) )/len(ae_old))
common = np.array(common)
len_crys    = np.array(len_crys)

np.std(np.array([len(ae_old), common], dtype=object))/len(ae_old) * 100

### Comparing slice 62

In [ ]:
timestep   = -1
filename   = '../../02_processed_data/Training_data/Other_Dataset/Randomslice56/AE_labels.sav'
ae_labels  = np.array(pickle.load(open(filename, 'rb')))[timestep]
#frac_ae    = crystal_environment(ae_labels, grid)

pathlist = Path('../../02_processed_data/Training_data/Other_Dataset/').glob('Random*/AE_labels.sav')
common  = []
len_crys     = []
for path in pathlist:
    # because path is object not string
    filename   = str(path)
    print(filename)
    labe1s_new = np.array(pickle.load(open(filename, 'rb')))[timestep]
    ae_old     = np.where(ae_labels  == 1)[0]
    ae_new     = np.where(labe1s_new == 1)[0]
    len_crys.append(len(ae_new))
    #p_ff.append(weird_division(ae_old, ae_new))
    common.append(sum(np.in1d(ae_old, ae_new)))
    #print(np.std(np.array([len(ae_old), random_common[i]]) )/len(ae_old))
common = np.array(common)
len_crys    = np.array(len_crys)

np.std(np.array([len(ae_old), common], dtype=object))/len(ae_old) * 100

### Comparing slice 68

In [ ]:
timestep   = -1
filename   = '../../02_processed_data/Training_data/Other_Dataset/Randomslice68/AE_labels.sav'
ae_labels  = np.array(pickle.load(open(filename, 'rb')))[timestep]
#frac_ae    = crystal_environment(ae_labels, grid)

pathlist = Path('../../02_processed_data/Training_data/Other_Dataset/').glob('Random*/AE_labels.sav')
common  = []
len_crys     = []
for path in pathlist:
    # because path is object not string
    filename   = str(path)
    print(filename)
    labe1s_new = np.array(pickle.load(open(filename, 'rb')))[timestep]
    ae_old     = np.where(ae_labels  == 1)[0]
    ae_new     = np.where(labe1s_new == 1)[0]
    len_crys.append(len(ae_new))
    #p_ff.append(weird_division(ae_old, ae_new))
    common.append(sum(np.in1d(ae_old, ae_new)))
    #print(np.std(np.array([len(ae_old), random_common[i]]) )/len(ae_old))
common = np.array(common)
len_crys    = np.array(len_crys)

np.std(np.array([len(ae_old), common], dtype=object))/len(ae_old) * 100

### Comparing point 20k

In [ ]:
timestep   = -1
filename   = '../../02_processed_data/Training_data/Other_Dataset/Randompoints20k/AE_labels.sav'
ae_labels  = np.array(pickle.load(open(filename, 'rb')))[timestep]
#frac_ae    = crystal_environment(ae_labels, grid)

pathlist = Path('../../02_processed_data/Training_data/Other_Dataset/').glob('Random*/AE_labels.sav')
common  = []
len_crys     = []
for path in pathlist:
    # because path is object not string
    filename   = str(path)
    print(filename)
    labe1s_new = np.array(pickle.load(open(filename, 'rb')))[timestep]
    ae_old     = np.where(ae_labels  == 1)[0]
    ae_new     = np.where(labe1s_new == 1)[0]
    len_crys.append(len(ae_new))
    #p_ff.append(weird_division(ae_old, ae_new))
    common.append(sum(np.in1d(ae_old, ae_new)))
    #print(np.std(np.array([len(ae_old), random_common[i]]) )/len(ae_old))
common = np.array(common)
len_crys    = np.array(len_crys)

np.std(np.array([len(ae_old), common], dtype=object))/len(ae_old) * 100